<a href="https://colab.research.google.com/github/fikird/fikird/blob/main/crf_amharic_pos_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
train_data = pd.read_csv('/content/train_set.tsv', sep='\t')
test_data = pd.read_csv('/content/test_set.tsv', sep='\t')
dev_data = pd.read_csv('/content/dev_set.tsv', sep='\t')


In [ ]:
train_data.loc[0]=['ስህፈት','N']
dev_data.loc[0]=['ወደእነዚያ','PRONP']
test_data.loc[0]=['አላህ','N']

In [ ]:
train_data.columns=['Word','Tag']
test_data.columns=['Word','Tag']
dev_data.columns=['Word','Tag']

creates features or columns

In [ ]:
def word_features(sent, i):
    """
    Extract features from a given sentence at index i.

    Args:
        sent (list): A list of tuples where each tuple contains (word, pos_tag).
        i (int): The index of the word in the sentence to extract features for.
        tagger (CRFTagger): An instance of a CRFTagger.

    Returns:
        dict: A dictionary containing various features of the word at index i.
    """
    word = sent['Word'][i]
    pos = sent['Tag'][i]

    # First word
    if i == 0:
        prevword = '<START>'
        prevpos = '<START>'
    else:
        prevword = sent['Word'][i - 1]
        prevpos = sent['Tag'][i - 1]

    # Second word
    if i == 0 or i == 1:
        prev2word = '<START>'
        prev2pos = '<START>'
    else:
        prev2word = sent['Word'][i - 2]
        prev2pos = sent['Tag'][i - 2]

    # Last word
    if i == len(sent) - 1:
        nextword = '<END>'
        nextpos = '<END>'
    else:
        nextword = sent['Word'][i + 1]
        nextpos = sent['Tag'][i + 1]

    return {
        'word': word,
        'pos': pos,  # Include the POS tag as a feature
        'prevword': prevword,
        'prevpos': prevpos,
        'nextword': nextword,
        'nextpos': nextpos,
        'prev2word': prev2word,
        'prev2pos': prev2pos,
    }
def extract(dataset):
    """
    Extract features for each sentence in the dataset.

    Args:
        dataset (pd.DataFrame): The dataset containing the sentences.

    Returns:
        list: A list of dictionaries, where each dictionary contains the features for a sentence.
    """
    features = []
    for i in  range(len(dataset)):
        features.append(word_features(dataset, i))
    return features

In [ ]:
val=extract( train_data)
test=extract(dev_data)
train=extract(test_data)

splits x and y

In [ ]:
def xy(data):
    X=[]
    y=[]
    for features in data:
        X.append({key: value for key, value in features.items() if key != 'pos'})
        y.append(features['pos'])
#     y = [{label} for label in y]
    return X, y

In [ ]:
val_X, val_y=xy(val)
test_X,test_y=xy(test)
train_X,train_y=xy(train)

make list of lists of features

In [ ]:
def prepare_data(data, labels):
    features = []
    label_sequences = []

    current_features = []
    current_labels = []
    for i in range(len(data)):
        # Create feature dictionary
        feature_dict = {
            'word': data[i]['word'],
            'prevword': data[i]['prevword'],
            'nextword': data[i]['nextword'],
            'prevpos': data[i]['prevpos'],
            'nextpos': data[i]['nextpos'],
            'prev2word': data[i]['prev2word'],
            'prev2pos': data[i]['prev2pos']
        }

        current_features.append(feature_dict)

        # Append the corresponding tag (label)
        if i < len(labels):
            current_labels.append(labels[i])

        # If it's the last token or end of sentence condition is met, save the sequence
        if i == len(data) - 1 or data[i + 1]['nextword'] == '<END>':
            features.append(current_features)
            label_sequences.append(current_labels)
            current_features = []
            current_labels = []

    return features, label_sequences

# Prepare your data
train_X, train_y = prepare_data(train_X, train_y)
test_X, test_y = prepare_data(test_X, test_y)
val_X, val_y = prepare_data(val_X, val_y)
# Now X is a list of lists of feature dictionaries, and y is a list of lists of labels

# declare a crf model

In [ ]:
import sklearn_crfsuite

# Initialize the CRF model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

ModuleNotFoundError: No module named 'sklearn_crfsuite'

# train

In [ ]:
crf.fit(train_X, train_y)

# validation and testing

In [ ]:
# Validate on validation set (if applicable)
y_val_pred = crf.predict([val_X])  # Ensure this is structured correctly

# Evaluate performance using F1 score
from sklearn_crfsuite import metrics

f1_score = metrics.flat_f1_score(val_y, y_val_pred, average='weighted')
print(f"Validation F1 Score: {f1_score:.4f}")

# Test on test set
y_test_pred = crf.predict([test_X])  # Ensure this is structured correctly

# Evaluate performance on test set
test_f1_score = metrics.flat_f1_score(test_y, y_test_pred, average='weighted')
print(f"Test F1 Score: {test_f1_score:.4f}")

# rendomized search for parameters

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(flat_f1_score, average='weighted')

rs = RandomizedSearchCV(crf, params_space, cv=3, scoring=f1_scorer)
rs.fit()